In [66]:
#set up autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import polars as pl
from pathlib import Path

import importlib  
fa_etl = importlib.import_module("fa-etl")

In [68]:
county = "36061"
path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"

In [69]:
taxhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/TaxHist{county}.parquet")
valhist = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_ValHist{county}.parquet")
annual = pl.read_parquet(path_to_dir+f"dev/{county}/staging/Prop{county}.parquet")
sales = pl.read_parquet(path_to_dir+f"dev/{county}/staging/ranked_Deed{county}.parquet")

In [70]:
valhist.filter(
    pl.col('Year') > 2017
)

PropertyID,AssdTotalValue,AssdYear,MarketTotalValue,MarketValueYear,ApprTotalValue,ApprYear,TaxableYear,Year,Value,AssessmentUsed,RecentValueByYear
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,u32
91849032,672438,2018,4558000,2018,0,null,null,2018,4558000,"""Market""",1
91849011,109355,2018,243012,2018,0,null,null,2018,243012,"""Market""",1
91849026,616712,2018,3799000,2018,0,null,null,2018,3799000,"""Market""",1
91849025,232401,2018,516447,2018,0,null,null,2018,516447,"""Market""",1
91849024,576595,2018,3799000,2018,0,null,null,2018,3799000,"""Market""",1
91848291,41919,2018,93153,2018,0,null,null,2018,93153,"""Market""",1
91848380,41919,2018,93153,2018,0,null,null,2018,93153,"""Market""",1
91848104,108484,2018,241076,2018,0,null,null,2018,241076,"""Market""",1
91847856,73548,2018,163440,2018,0,null,null,2018,163440,"""Market""",1


In [71]:
merged = pl.read_parquet(f"/Users/claireboyd/internships/mansueto/firstamerican-etl/dev/{county}/unified/merged.parquet")

In [72]:
merged.shape

(2456743, 28)

In [73]:
merged.head(10)

PropertyID,AssdTotalValue,AssdYear,MarketTotalValue,MarketValueYear,ApprTotalValue,ApprYear,TaxableYear,Year,Value,AssessmentUsed,RecentValueByYear,PropertyClassID,FATimeStamp,SitusLatitude,SitusLongitude,SitusFullStreetAddress,SitusCity,SitusState,SitusZIP5,FIPS,SitusCensusTract,SitusCensusBlock,SitusGeoStatusCode,FIPS_SitusCensusTract,SaleAmt,TaxAmt,TaxAmtAdjusted
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,u32,str,date,f64,f64,str,str,str,str,str,str,str,str,str,i64,i64,f64
91849011,109355,2018,243012,2018,0,null,null,2018,243012,"""Market""",1,"""R""",2024-02-07,40.744385,-73.984104,"""39 E 29TH ST A…","""NEW YORK""","""NY""","""10016""","""36061""","""007400""","""2001""","""B""","""36061_7400""",null,954788,9547.88
91849025,232401,2018,516447,2018,0,null,null,2018,516447,"""Market""",1,"""R""",2024-02-07,40.744385,-73.984104,"""39 E 29TH ST A…","""NEW YORK""","""NY""","""10016""","""36061""","""007400""","""2001""","""B""","""36061_7400""",null,2029124,20291.24
91848291,41919,2018,93153,2018,0,null,null,2018,93153,"""Market""",1,"""R""",2024-02-07,40.745345,-73.999298,"""312 W 23RD ST …","""NEW YORK""","""NY""","""10011""","""36061""","""009300""","""2001""","""B""","""36061_9300""",null,502220,5022.2
91848380,41919,2018,93153,2018,0,null,null,2018,93153,"""Market""",1,"""R""",2024-02-07,40.745345,-73.999298,"""312 W 23RD ST …","""NEW YORK""","""NY""","""10011""","""36061""","""009300""","""2001""","""B""","""36061_9300""",null,0,0.0
91848104,108484,2018,241076,2018,0,null,null,2018,241076,"""Market""",1,"""R""",2024-02-07,40.750965,-73.982198,"""425 5TH AVE AP…","""NEW YORK""","""NY""","""10016""","""36061""","""008200""","""2004""","""B""","""36061_8200""",null,1205772,12057.72
91847856,73548,2018,163440,2018,0,null,null,2018,163440,"""Market""",1,"""R""",2024-02-07,40.750965,-73.982198,"""425 5TH AVE AP…","""NEW YORK""","""NY""","""10016""","""36061""","""008200""","""2004""","""B""","""36061_8200""",null,791236,7912.36
91847850,1093365,2018,3625000,2018,0,null,null,2018,3625000,"""Market""",1,"""R""",2024-02-07,40.739371,-73.989025,"""24 E 21ST ST #…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""1000""","""B""","""36061_5200""",null,13985640,139856.4
91847993,254799,2018,566219,2018,0,null,null,2018,566219,"""Market""",1,"""R""",2024-02-07,40.729882,-74.007313,"""421 HUDSON ST …","""NEW YORK""","""NY""","""10014""","""36061""","""006900""","""2005""","""B""","""36061_6900""",null,2831008,28310.08
91847796,297186,2018,660413,2018,0,null,null,2018,660413,"""Market""",1,"""R""",2024-02-07,40.739961,-73.990361,"""141 5TH AVE AP…","""NEW YORK""","""NY""","""10010""","""36061""","""005200""","""2000""","""B""","""36061_5200""",null,3180028,31800.28


In [74]:
merged.filter([
    pl.col("SaleAmt").is_not_null(),
    pl.col("Year") > 2017
]).select(
    ['PropertyID','Year','Value','SaleAmt', 'TaxAmt', 'TaxAmtAdjusted']
).with_columns([
    (pl.col('TaxAmt')/pl.col('Value')).alias("ApproxTaxRate")
])

PropertyID,Year,Value,SaleAmt,TaxAmt,TaxAmtAdjusted,ApproxTaxRate
i64,i64,i64,i64,i64,f64,f64
91847870,2018,261504,1580000,1310424,13104.24,5.011105
91849008,2018,1462000,4400000,3114052,31140.52,2.129995
91848046,2018,158123,712775,773352,7733.52,4.890825
91846549,2018,331800,2475000,1527372,15273.72,4.603291
91847537,2018,285721,1625000,1496148,14961.48,5.236395
91848333,2018,793100,7400000,3958776,39587.76,4.991522
91846649,2018,487975,3100000,2518932,25189.32,5.16201
91847215,2018,665110,2825000,3327036,33270.36,5.002234
91846134,2022,1595704,12500000,8808528,88085.28,5.520152
